Creating your own pix2pix dataset
=================================



## Installation requirements

To run this notebook you might need to install some new Python packages. To do so, open a terminal and first make sure your environment is active
```
conda activate dmlap
```
and then
```
conda install -c conda-forge pycairo opencv scikit-image 
conda install -c conda-forge face_recognition
pip install pyglet
```

If you have not done so already, you should also need to install the [py5canvas](https://github.com/colormotor/py5canvas) module. To do so use 
```
pip install git+https://github.com/colormotor/py5canvas.git
```

### Updating py5canvas
If you already installed py5canvas, you will need to updated it to the latest version. To do so use 
```
pip install --upgrade  --force-reinstall --no-deps git+https://github.com/colormotor/py5canvas.git
```

In [ ]:
## Modules
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, transform
import cv2
import glob
from tqdm.auto import tqdm
import random

## Setting up 
Set your directories and the dataset specifics

-   `target_path` defines where your **target** images are located.
-   `source_path` defines where your **source** images are located, if you already have these. Otherwise, set this to an empty string `''`.
-   `dataset_path` defines where your pix2pix dataset will be saved.
-   `is_input_pix_to_pix` set this to `True` if the input dataset already consists of an source and target pairs. This will be the case if you want to modify an existing pix2pix dataset. In this case we need to extract only the target.
-   `input_target_index` if we are manipulating a dataset that is already a pix2pix dataset, this defines whether the target image is to the left (`0`) or to the right (`1`).

Note you will have to put exactly the path to your image directories here, this code does not recursively search for images. Also note that the most common use case for this system will be with you providing an dataset of targets (desired outputs) that you will process to create the corresponding inputs (e.g. with edge detection or finding face landmarks). In that case you should not worry about the `source_path` directory below.

Here, by default we will load the &ldquo;Face 2 comics&rdquo; dataset. Download the dataset from [https://www.kaggle.com/datasets/defileroff/comic-faces-paired-synthetic](https://www.kaggle.com/datasets/defileroff/comic-faces-paired-synthetic), unzip, and place the `face2comics_v1.0.0_by_Sxela` direcory in your dataset directory. This is already a &ldquo;pix2pix-friendly&rdquo; dataset consisting, however, of pairs of images that are separated. We will use the images to create an &ldquo;Edges to comics&rdquo; dataset, where we apply edge detection to a subset of the source images and leave the corresponding comic version unchanged.



In [ ]:
import os

target_path = './datasets/face2comics_v1.0.0_by_Sxela/comics/'
source_path = './datasets/face2comics_v1.0.0_by_Sxela/face/'  # Only used if we already have source image examples
dataset_path = './datasets/edge2comics'
max_images = 500
is_input_pix_to_pix = False
input_target_index = 1

# Uncomment and adjust paths to perform face detection as the source
# target_path = './datasets/edges2rembrandt'
# source_path = ''
# dataset_path = './datasets/landmarks2rembrandt'
# max_images = 500
# is_input_pix_to_pix = True
# input_target_index = 1

The code above also contains a commented section with paths for the case in which you already have a pix2pix dataset with 512x256 images, and you want to replace the input with a custom one. Later in the code there is a commented section that will identify face landmarks in the targets of the dataset (the right image) and use these as an input. For this specific example to work, it is expected that you download the ["rembrandt pix2pix dataset"](https://www.kaggle.com/datasets/grafstor/rembrandt-pix2pix-dataset/code) and unzip the images into a "edges2rembrandt" folder inside the dataset folder relative to this notebook.

## Load the images to process



Now let&rsquo;s load our target images, and optionally our source images if we have set the `source_path` directory



In [ ]:

def load_image(path):
    w, h = (256, 256)
    if is_input_pix_to_pix: # In case we are already loading a pix2pix image
        w, h = (512, 256)
    img = io.imread(path) #image.load_img(path, target_size=size)
    img = transform.resize(img, (h, w), anti_aliasing=True)
    # If we are loading a pix2pix dataset just extract the target
    if is_input_pix_to_pix:
        if input_target_index==0:
            img = img[:,:h,:]
        else:
            img = img[:,h:,:]
    return (img*255).astype(np.uint8)

def load_images_in_path(path):
    files = glob.glob(path + '/*')
    images = []
    if max_images:
        n = len(files)
        files = files[:max_images]
        print('%d of %d images'%(len(files), n))
    else:
        print('%d images'%len(files))
    for imgfile in tqdm(files): #, desc='Loading images in ' + path):
        img = load_image(imgfile)
        images.append(img)
    return images

print('Loading targets')
target_images = load_images_in_path(target_path)
if source_path:
    print('Loaded sources')
    source_images = load_images_in_path(source_path)


In [ ]:
target_images[0].shape

## Define our transformation



The code below has a number of transformations already setup for you. These are:

-   `apply_canny_cv2` Applys Canny edge detection by using OpenCV. You can set two parameters (thresholds between 0 and 255) that will determine the result of the edge detection: `thresh1` and `thresh2`. Experiment with these values to adjust the results to your liking. Additional details can be seen [here](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de).
-   `apply_canny_skimage` Applys Canny edge detection by using [scikit-image](https://scikit-image.org). You can set one parameter, `sigma` that determines the number of edges. In general, a higher number will produce less edges. See [this](https://scikit-image.org/docs/stable/auto_examples/edges/plot_canny.html) for additional details.
-   `apply_face_landmarks` Finds face landmarks in an image by using [face_recognition](https://pypi.org/project/face-recognition/) and uses the Canvas API to draw these as polygons. Note that this function will fail if the face detector cannot find a face in the image. The code is set up so the image won't be included in the generated dataset if face detection fails.
-   `load_source` assumes the `source_path` directory has been set, and simply loads an image from the directory leaving it unchanged. Use this if you wish to convert a dataset made of separate images, into a dataset consisting of 512X256 sized images (the format expected by the model provided in class). 
-   `transformation(transformation type)` takes an image from the directory specified in `source_path`. Note that this is a function call (it returns a function). You can optionally set an additional transformation to the image by setting the `transform_func` parameter to one of the functions above. For example setting `transform=apply_canny_skimage` will apply the Canny edge detection algorithm to the loaded source image. By default, no transormation is applied.

Set the `image_transformation` in the code below to the function that describes the transformation you want to apply.
If you feel confident, you can extend this to other image transformations by duplicating one of the functions and adapting it to your needs.


In [ ]:
from skimage import feature, filters

def apply_canny_cv2(index, img, thresh1=160, thresh2=250):
    import cv2
    invert = False
    grayimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(grayimg, thresh1, thresh2)
    if invert:
        edges = cv2.bitwise_not(edges)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

def apply_canny_skimage(index, img, sigma=1.5):
    import cv2
    from skimage import feature
    invert = False
    grayimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = (feature.canny(grayimg, sigma=sigma)*255).astype(np.uint8)
    if invert:
        edges = cv2.bitwise_not(edges)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

def apply_face_landmarks(index, img, stroke_weight=2):
    from py5canvas import canvas
    import face_recognition
    
    c = canvas.Canvas(256, 256)
    c.background(0)
    landmarks = face_recognition.face_landmarks(img)

    if not landmarks:
        print('Failed to find landmarks')
        return None
    c.stroke_weight(stroke_weight)
    c.no_fill()
    c.stroke(255)
    for points in landmarks[0].values():
        c.polyline(points)
    return c.get_image()

def load_source(index, img):
    if not source_path:
        raise ValueError("Source path must be specified")
    return source_images[index]

def do_nothing(index, img):
    return img

# As it is, this version loads an image from the source_image directory and applies the Canny edge detection
# algorithm to it. Set transform=None if you just want to load that image without processing
def transformed_source(transform):
    if not source_path:
        raise ValueError("Source path must be specified")
    def func(index, img):
        return transform(index, source_images[index])
    return func

# This takes an image in the `source_path`` directory and applys edge detection to it,
# creating a new source
#source_func = transformed_source(apply_canny_skimage)

# This applies edge detection to the target image, making it the source to the tranformation you want to apply. If you are only working with a single folder of images that you
# image_tranformation = apply_canny_skimage

# This tries to find face landmarks in the target and creates a new source
source_func = apply_face_landmarks # <- Use this for face detection

# By default, just pass through the target image with no transformation
target_func = do_nothing

# Show an example
i = 1
img = target_images[i]
src = source_func(i, img)
plt.figure()
plt.subplot(1, 2, 1)
if src is not None:
plt.subplot(1, 2, 2)
plt.imshow(img)
plt.show()


## Create the dataset!



Here we loop through all the target images, generate the source image and stitch these together into a single image. The input image directory might contain more than the desired number of images. If we want to process a lower number, set the `num_images` variable to a non-zero value.



In [ ]:

# target_index = 1 # You can redefine this if you wish for example to flip target and source

num_images = 500
shuffle = True
image_indices = list(range(len(target_images)))
if shuffle:
    random.shuffle(image_indices)
if num_images != 0:
    image_indices = image_indices[:num_images]

os.makedirs(dataset_path, exist_ok=True)

index = 1
for i in tqdm(image_indices, desc='Saving dataset to ' + dataset_path):
    target = target_func(i, target_images[i])
    source = source_func(i, target) 
    if source is None:
        print('Failed to transform image %d of %d'%(i+1, len(image_indices)))
        continue

    if target_index==1:
        combined = np.hstack([source, target])
    else:
        combined = np.hstack([target, source])
    io.imsave(os.path.join(dataset_path, '%d.png'%(index)), combined)
    index += 1